# Installation et Import des Bibliothèques
Installation et import des bibliothèques nécessaires: transformers, torch, pandas, numpy. Configuration des modèles BERT multilingue et BERT standard.

In [1]:


# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForTokenClassification, BertForMaskedLM

# Configurer les modèles BERT
tokenizer_multilingual = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_multilingual = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')

tokenizer_standard = BertTokenizer.from_pretrained('bert-base-cased')
model_standard = BertForMaskedLM.from_pretrained('bert-base-cased')

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an

# Chargement des Modèles Transformers
Chargement de bert-base-multilingual-cased pour le POS tagging et bert-base-cased pour la sélection des mots. Configuration des tokenizers et des pipelines.

In [2]:
# Chargement des Modèles Transformers

# Importer les bibliothèques nécessaires
from transformers import pipeline

# Configurer les pipelines pour l'analyse grammaticale et la sélection des mots
pos_tagger = pipeline('ner', model=model_multilingual, tokenizer=tokenizer_multilingual)
word_replacer = pipeline('fill-mask', model=model_standard, tokenizer=tokenizer_standard)

# Identification des Verbes avec POS Tagging
Utilisation du modèle BERT multilingue pour identifier les verbes dans les proverbes modifiés via l'analyse grammaticale (POS tagging).

In [10]:
# Fonction pour identifier les verbes dans les proverbes modifiés
def identify_verbs(proverbs):
    verb_positions = []
    for proverb in proverbs:
        masked_proverb = proverb['Masked']
        tokens = pos_tagger(masked_proverb)
        # Print tokens to see the actual entity labels
        print(tokens)
        verbs = [token for token in tokens if 'VERB' in token['entity']]
        verb_positions.append(verbs)
    return verb_positions

# Proverbes modifiés pour les tests
tests = [{'Masked': 'Les actes valent mieux que les paroles'},
         {'Masked': 'Les actes mangent mieux que les paroles'}]
    
# Identifier les verbes dans les proverbes modifiés
verb_positions = identify_verbs(tests)

# Afficher les positions des verbes identifiés
verb_positions

[{'entity': 'LABEL_1', 'score': np.float32(0.57922167), 'index': 1, 'word': 'Les', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.68530345), 'index': 2, 'word': 'actes', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.6509647), 'index': 3, 'word': 'vale', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.67500263), 'index': 4, 'word': '##nt', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.63514495), 'index': 5, 'word': 'mieux', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.67442673), 'index': 6, 'word': 'que', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.6825574), 'index': 7, 'word': 'les', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score': np.float32(0.7003117), 'index': 8, 'word': 'paroles', 'start': None, 'end': None}]
[{'entity': 'LABEL_1', 'score': np.float32(0.5686393), 'index': 1, 'word': 'Les', 'start': None, 'end': 

[[], []]

# Sélection du Meilleur Mot
Utilisation du modèle BERT pour calculer la probabilité de chaque mot candidat dans le contexte du proverbe et sélectionner le plus approprié.

In [ ]:
# Sélection du Meilleur Mot

# Fonction pour remplacer les verbes identifiés par les mots candidats et sélectionner le meilleur mot
def correct_proverbs(proverbs, verb_positions):
    corrected_proverbs = []
    for i, proverb in enumerate(proverbs):
        masked_proverb = proverb['Masked']
        word_list = proverb['Word_list']
        best_proverb = masked_proverb
        best_score = float('-inf')
        
        for word in word_list:
            # Remplacer le verbe par le mot candidat
            for verb in verb_positions[i]:
                start, end = verb['start'], verb['end']
                masked_proverb = masked_proverb[:start] + '[MASK]' + masked_proverb[end:]
                masked_proverb = masked_proverb.replace('[MASK]', word, 1)
                
                # Calculer la probabilité du proverbe avec le mot candidat
                inputs = tokenizer_standard(masked_proverb, return_tensors='pt')
                outputs = model_standard(**inputs)
                logits = outputs.logits
                mask_token_index = torch.where(inputs.input_ids == tokenizer_standard.mask_token_id)[1]
                mask_token_logits = logits[0, mask_token_index, :]
                top_token = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()
                
                # Vérifier si le mot candidat est le meilleur
                if word in tokenizer_standard.convert_ids_to_tokens(top_token):
                    score = mask_token_logits[0, tokenizer_standard.convert_tokens_to_ids(word)].item()
                    if score > best_score:
                        best_score = score
                        best_proverb = masked_proverb
        
        corrected_proverbs.append(best_proverb)
    return corrected_proverbs

# Corriger les proverbes
corrected_proverbs = correct_proverbs(tests, verb_positions)

# Afficher les proverbes corrigés
corrected_proverbs

# Évaluation et Analyse des Résultats
Évaluation de la performance du système, calcul des métriques et analyse comparative avec l'approche du TP1. Analyse qualitative des erreurs.

In [ ]:
# Évaluation et Analyse des Résultats

# Fonction pour évaluer la performance du système
def evaluate_performance(corrected_proverbs, original_proverbs):
    correct_count = 0
    total_count = len(original_proverbs)
    
    for corrected, original in zip(corrected_proverbs, original_proverbs):
        if corrected == original['Proverb']:
            correct_count += 1
    
    accuracy = correct_count / total_count
    return accuracy

# Calculer la précision du système
accuracy = evaluate_performance(corrected_proverbs, tests)

# Afficher la précision
print(f"Précision du système: {accuracy * 100:.2f}%")

# Analyse comparative avec l'approche du TP1
# (Cette section est un exemple et doit être remplie avec les résultats réels du TP1)
accuracy_tp1 = 0.85  # Exemple de précision du TP1

print(f"Précision de l'approche du TP1: {accuracy_tp1 * 100:.2f}%")
print(f"Amélioration de la précision: {(accuracy - accuracy_tp1) * 100:.2f}%")

# Analyse qualitative des erreurs
def analyze_errors(corrected_proverbs, original_proverbs):
    errors = []
    for corrected, original in zip(corrected_proverbs, original_proverbs):
        if corrected != original['Proverb']:
            errors.append({
                'Masked': original['Masked'],
                'Corrected': corrected,
                'Original': original['Proverb']
            })
    return errors

# Obtenir les erreurs
errors = analyze_errors(corrected_proverbs, tests)

# Afficher les erreurs
for error in errors:
    print(f"Proverbe masqué: {error['Masked']}")
    print(f"Proverbe corrigé: {error['Corrected']}")
    print(f"Proverbe original: {error['Original']}")
    print("---")

# Analyse des erreurs
# (Cette section est un exemple et doit être remplie avec une analyse réelle des erreurs)
print("Analyse des erreurs:")
print("1. Les erreurs peuvent être dues à une mauvaise identification des verbes.")
print("2. Les erreurs peuvent également être dues à une mauvaise sélection des mots candidats.")
print("3. Certaines erreurs peuvent être dues à des proverbes moins courants ou à des variations régionales.")